# Intro
In this notebook, we briefly demonstrate how you can use pre-built models from the [open model hub](https://objectiv.io/docs/modeling/Objectiv/bach_open_taxonomy.ModelHub/#bach_open_taxonomy.ModelHub) in conjunction with our [modeling library](https://objectiv.io/docs/modeling/) to quickly build model stacks to answer common product analytics questions.

This example uses real, unaltered data that was collected from objectiv.io with Objectiv’s Tracker. All models in the open model hub are compatible with datasets that have been validated against the [open analytics taxonomy](https://objectiv.io/docs/taxonomy/).

For an overview of all available models, check out the [open model hub docs](https://objectiv.io/docs/modeling/Objectiv/bach_open_taxonomy.ModelHub/#bach_open_taxonomy.ModelHub).

In [9]:
from bach_open_taxonomy import ObjectivFrame
from bach import display_sql_as_markdown

# Instantiate the data object
As a first step, the Objectiv Frame object is instantiated. The Objectiv Frame is an extension to Bach DataFrame to use specifically for data that was collected with Objectiv’s Tracker. Bach is Objectiv’s data modeling library. With Bach, you can use familiar Pandas-like DataFrame operations in your notebook. It uses a SQL abstraction layer that enables models to run on the full dataset, and you can output models to SQL with a single command.

The Objectiv Bach api is heavily inspired by the pandas api. We believe this provides a great, generic interface to handle large amounts of data in a python environment while supporting multiple data stores. For more details on Objectiv Bach, visit the docs.

It loads the data as stored by the Objectiv Tracker, makes a few transformations, and sets the right data types.

This object points to the data on which the models from the open model hub will be applied. The `time_aggregation` parameter determines the standard timeframe that is used with aggregation functions from the model hub. Ie. 'YYYY-MM-DD' means that days are used for the time aggregation. Only data starting at `start_date` is used for all following operations.

**Note**  
All operation and models in this notebook are run directly on entire data set in the SQL database using Bach. While the api resembles pandas, pandas is _not_ used for any the operations and calculations.

In [11]:
of = ObjectivFrame.from_objectiv_data(time_aggregation='YYYY-MM-DD', start_date='2021-11-16')

OperationalError: (psycopg2.OperationalError) connection to server at "localhost" (::1), port 5432 failed: Connection refused
	Is the server running on that host and accepting TCP/IP connections?
connection to server at "localhost" (127.0.0.1), port 5432 failed: Connection refused
	Is the server running on that host and accepting TCP/IP connections?

(Background on this error at: https://sqlalche.me/e/14/e3q8)

# Using the open model hub
The ModelHub contains a growing collection of open-source, free to use data models that you can take, chain and run to quickly build highly specific model stacks for product analysis and exploration. It includes models for a wide range of typical product analytics use cases. The source is available for all models and you're free to make any changes to them. 

All models are in the `.model_hub` namespace of the Objectiv Frame. You can also use `.mh` as a shorthand. 

The model hub has two main type of functions: `map` and `aggregate`. 
* `map` functions always return a series with the same shape and index as the Objectiv Frame they originate from. This ensures they can be added as a column to that Objectiv Frame.
* `aggregate` fuctions return aggregated data in some form from the Objectiv Frame. Can also be accessed with `agg`.

Additionally the Objectiv Frame can be filtered with the `filter` method. `map` functions that return SeriesBoolean can be used with the `filter` method.

This notebook demonstrates how to use the model hub by showcasing a selection of the models from the model hub.

## A simple aggregation model
Calculating the unique users is one of the basic models in the model hub. As it is an aggregation model, it is called with `.model_hub.aggregate.unique_users()`. It uses the time_aggregation that is set when the Objectiv Frame was instantiated. With `.head()` we immediately query the data to show the results. `.to_pandas()` can be used to use all results as a pandas object in python. These (and following) results are sorted descending, so we show the latest data first.

In [ ]:
users = of.model_hub.aggregate.unique_users()
users.sort_index(ascending=False).head(10)

## Using `map` with the model hub & combining models
This example shows how you use map to label users as a new user. This uses *time_aggregation*. As *time_aggregation* was set to 'YYYY-MM-DD' it means all hits are labeled as new for the entire day in which the user had its first session.

In [5]:
is_new_user = of.model_hub.map.is_new_user()
is_new_user.head(10)

event_id
e5ff056b-7d8e-4fc5-97fe-f8dd1c8990ed    True
34e55001-d231-4dee-9951-163091383c9f    True
780f16dc-183a-4fed-bacb-cee556d14587    True
8cd817dd-16df-4c9b-9f04-0753c35b30a2    True
dfbdc5b0-f66b-477c-98de-682d843cc978    True
ce2e8484-d498-4190-a381-66b15c4beba7    True
82828e0f-9707-476b-b7ed-c1aa081eb96a    True
a4670ff1-8b5e-4253-9419-3b8b80d7ea89    True
3b684eaf-44b3-4bf6-b8f6-dbef3400eec6    True
3f393077-644a-42f6-bb06-a5a6b6f0ff4b    True
Name: is_new_user, dtype: bool

Or we can label conversion events. To do this we first have to define what a conversion is by setting the type of event and the location on the product at which this event was triggered with `add_conversion_event` (this is called the location stack, see [here](open-taxonomy-how-to.ipynb#global_contexts-&-location_stack) for info).

In [ ]:
of.add_conversion_event(location_stack=of.location_stack.json[{'id': 'Quickstart Guide', '_type': 'LinkContext'}:],
                        event_type='PressEvent',
                        name='quickstart_presses')
conversion_events = of.mh.map.is_conversion_event('quickstart_presses')
conversion_events.head(10)

### Map, filter, aggregate
As the map functions above retured a SeriesBoolean, they can be used in the model hub combined with filter and aggregation models. We use the same aggregation model we showed earlier (`unique_users`), but now with the filter `conversion_events` applied. This gives the unique converted users per day.

In [ ]:
of.model_hub.filter(conversion_events).model_hub.aggregate.unique_users().head(10)

Similarly, we can use other aggregation models from the model hub. In the example below, the average session duration is calculated for new users.

In [6]:
duration_new_users = of.model_hub.filter(is_new_user).model_hub.aggregate.session_duration()
duration_new_users.sort_index(ascending=False).head(10)

moment
2022-02-23   0 days 00:30:45.912500
2022-02-22   0 days 00:01:38.459273
2022-02-21   0 days 00:02:23.439643
2022-02-20   0 days 00:11:05.704100
2022-02-19   0 days 00:04:49.987333
2022-02-18   0 days 00:01:39.973000
2022-02-17   0 days 00:03:16.271568
2022-02-16   0 days 00:10:16.046000
2022-02-15   0 days 00:01:09.719857
2022-02-14   0 days 00:05:07.271000
Name: session_duration, dtype: timedelta64[ns]

### Combining model results
Results from aggregation models can be used together if they share the same index type (similar to pandas). In this example the share of new users per day is calculated.

In [7]:
new_user_share = of.mh.filter(is_new_user).mh.agg.unique_users() / of.mh.agg.unique_users()
new_user_share.sort_index(ascending=False).head(10)

moment
2022-02-23    0.250000
2022-02-22    0.500000
2022-02-21    0.666667
2022-02-20    0.727273
2022-02-19    0.333333
2022-02-18    0.480000
2022-02-17    0.640625
2022-02-16    0.555556
2022-02-15    0.380952
2022-02-14    0.571429
Name: unique_users, dtype: float64

### Chaining model filters

Filter model results can be chained, as the output is always an ObjectivFrame with the same columns but less or equal the number of rows as the initial ObjectivFrame. In this example we calculate the number of users that were new in a month and also that converted twice on a day.

In [ ]:
of.mh.filter(of.mh.map.is_new_user(time_aggregation = 'YYYY-MM'))\
  .mh.filter(of.mh.map.conversion_count(name='quickstart_presses')==2)\
  .mh.agg.unique_users()\
  .head()

## What's next?
There are several options on how to continue working with the data or using the results otherwise, i.e. for visualization.

### 1. Export models to sql
As mentioned, all operations and models performed on the ObjectivFrame are run on the SQL data base. Therefore it is possible to view all objects as an SQL statement. For the `new_user_share` results that was created this looks as follows:

In [10]:
# complex sql statement alert!
display_sql_as_markdown(new_user_share)

```sql
with "ExtractedContexts___dd72b41414a8325eaf393b125ae1f990" as (
    SELECT event_id,
            day,
            moment,
            cookie_id AS user_id,
            CAST(JSON_EXTRACT_PATH(value, 'global_contexts') AS jsonb) AS global_contexts,
            CAST(JSON_EXTRACT_PATH(value, 'location_stack') AS jsonb) AS location_stack,
            value->>'_type' AS event_type,
            CAST(JSON_EXTRACT_PATH(value, '_types') AS jsonb) AS stack_event_types
     FROM data
     where day >= '2022-02-02'
     ),
"session_starts_7f1b431488bd34d858e1d59857f3633a" as (
        select
            *,
            case when coalesce(
                extract(
                    epoch from (moment - lag(moment, 1)
                        over (partition by user_id order by moment, event_id))
                ) > 1800,
                true
            ) then true end as is_start_of_session
        from "ExtractedContexts___dd72b41414a8325eaf393b125ae1f990"
    ),
"session_id_and_start_7f1b431488bd34d858e1d59857f3633a" as (
        select
            *,
            -- generates a session_start_id for each is_start_of_session
            case
                when is_start_of_session then
                    row_number() over (partition by is_start_of_session order by moment, event_id)
            end as session_start_id,
            -- generates a unique number for each session, but not in the right order.
            count(is_start_of_session) over (order by user_id, moment, event_id) as is_one_session
        from session_starts_7f1b431488bd34d858e1d59857f3633a
    ),
"SessionizedData___7f1b431488bd34d858e1d59857f3633a" as (select
        *,
        -- populates the correct session_id for all rows with the same value for is_one_session
        first_value(
            session_start_id
        ) over (
            partition by is_one_session order by moment, event_id
        ) as session_id,
        row_number() over (partition by is_one_session order by moment, event_id) as session_hit_number
    from session_id_and_start_7f1b431488bd34d858e1d59857f3633a
    ),
"manual_materialize___6ba05d5f7eab48532772bc1dfce67bab" as (select "event_id", "day", "moment", "user_id", "global_contexts", "location_stack", "event_type", "stack_event_types", "session_id", "session_hit_number", (min("session_id") over (partition by to_char("moment", 'YYYY-MM-DD'), "user_id"  RANGE BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) = min("session_id") over (partition by "user_id"  RANGE BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING)) as "__filter" 
from "SessionizedData___7f1b431488bd34d858e1d59857f3633a" 
 
 
 
 
limit all 
),
"getitem_where_boolean___fa1e2bb19beb18745d214e370080f70c" as (select "event_id", "day", "moment", "user_id", "global_contexts", "location_stack", "event_type", "stack_event_types", "session_id", "session_hit_number" 
from "manual_materialize___6ba05d5f7eab48532772bc1dfce67bab" 
where "__filter" 
 
 
 
limit all 
),
"merge_left___d7f6c91ce58bd922a8b55b145fb16f0a" as (select to_char("moment", 'YYYY-MM-DD') as "moment", count(distinct "user_id") as "unique_users" 
from "getitem_where_boolean___fa1e2bb19beb18745d214e370080f70c" 
 
group by (to_char("moment", 'YYYY-MM-DD')) 
 
 
limit all 
),
"manual_materialize___793b90f8547872620650aa7d02164e1b" as (select to_char("moment", 'YYYY-MM-DD') as "moment", count(distinct "user_id") as "__other" 
from "SessionizedData___7f1b431488bd34d858e1d59857f3633a" 
 
group by (to_char("moment", 'YYYY-MM-DD')) 
 
 
limit all 
),
"merge_sql___3e867068dca9a706077ed78a49316888" as (
            select "l"."moment" as "moment", "l"."unique_users" as "unique_users", "r"."__other" as "__other", "r"."moment" as "moment__index"
            from "merge_left___d7f6c91ce58bd922a8b55b145fb16f0a" as l full outer
            join "manual_materialize___793b90f8547872620650aa7d02164e1b" as r on ("l"."moment" = "r"."moment")
            )
select (COALESCE("moment", "moment__index")) as "moment", (cast("unique_users" as float) / ("__other")) as "unique_users" 
from "merge_sql___3e867068dca9a706077ed78a49316888" 
 
 
 
 
limit all 
```

### 2. Export to metabase
Aggregation models can be exported to Metabase, to visualize and share your results. This is done for the unique new users:

In [ ]:
of.model_hub.to_metabase(of.mh.filter(is_new_user).mh.agg.unique_users(), config={'name': 'Unique New Users'})

The results can be viewed here: http://localhost:3000/dashboard/1-model-hub

### 3. Further data crunching using the Bach Modeling Library
Everything from the [Bach modeling library](https://objectiv.io/docs/modeling/) is available to ObjectivFrames. This makes the model hub and bach work seamlessly together.

In [ ]:
# label conversions and the number of time a user is converted in a session at a moment using the model hub.
of['is_conversion'] = of.model_hub.map.is_conversion_event(name='quickstart_presses')
of['conversion_count'] = of.model_hub.map.conversion_count(name='quickstart_presses')

# use Bach to do any supported operation using pandas syntax.
# select users that converted
converted_users = of[of.is_conversion].user_id.unique()
# select PressEvents of users that converted
of_selection = of[(of.event_type == 'PressEvent') &
                  (of.user_id.isin(converted_users))]
# calculate the number of PressEvents before conversion per session
presses_per_session = of_selection[of_selection.conversion_count == 0].groupby('session_id').session_hit_number.count()

Show the results, now the underlying query is executed.

In [ ]:
presses_per_session.head()

There is another notebook in the same folder that demonstrates what you can do with the Bach modeling library [open-taxonomy-how-to.ipynb](open-taxonomy-how-to.ipynb), or head over to the [api reference](https://objectiv.io/docs/modeling/reference) for a complete overview of the possibilities.

### 4. Export ObjectivFrame and model hub results to pandas DataFrame
ObjectivFrames and/or model hub results can always be exported to pandas. Since ObjectivFrame operation run on the full dataset in the SQL database, it is recommended to export to pandas if data small enough; ie by aggregation or selection.  
By exporting the data to pandas you can use all the options from pandas as well as pandas compatible ML packages.

We plot the previously calculated presses per session before conversion using pandas built-in plotting methods.

In [ ]:
# presses_per_session_pd is a pandas Series
presses_per_session_pd = presses_per_session.to_pandas()
presses_per_session_pd.hist()

This concludes the open model hub demo.

We hope you’ve gotten a taste of the power and flexibility of the open model hub to quickly answer common product analytics questions. You can take it a lot further and build highly specific model stacks for in-depth analysis and exploration.

For a complete overview of all available and upcoming models, check out the [model hub docs](https://objectiv.io/docs/modeling/Objectiv/bach_open_taxonomy.ModelHub).